In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import Proxy

# = = = = = = = = = = = = = = =

from openpyxl import load_workbook

wb = load_workbook('./crawler.xlsx')
ws = wb.active

for column in range(1, ws.max_column+1):
    if ws.cell(1, column).value == 'Src':
        src_index = column
    elif ws.cell(1, column).value == 'Pic':
        pic_index = column

print('Src索引：' + str(src_index))
print('Pic索引：' + str(pic_index))
print()     
print('总数量：' + str(ws.max_row+1))
print()     

# = = = = = = = = = = = = = = =

import requests
from io import BytesIO
from openpyxl.drawing.image import Image

from openpyxl.utils import get_column_letter

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(2, ws.max_row+1):
    work.put_nowait(i)

def crawler():
    global ws
    
    while not work.empty():
        i = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                src = ws.cell(i, src_index).value
                
                if src == '':
                    status = 'ok'
                    
                    break
                    
                pic = Image(BytesIO(requests.get(src, proxies=Proxy.get_Proxy_Requests()).content))
                
                # = = = = = = = = = = = = = = =
                
                if pic.width >= pic.height:
                    pic.height = 50.0 / pic.width * pic.height
                    pic.width = 50.0
                else:
                    pic.width = 50.0 / pic.height * pic.width
                    pic.height = 50.0

                # = = = = = = = = = = = = = = =
                
                ws.add_image(pic, get_column_letter(pic_index) + str(i))

                status = 'ok'
                
                break
   
            except:
                time.sleep(0.5)
                continue

        print(src + '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
wb.save('./crawler-pic.xlsx')
print()
print('爬虫结束')

Src索引：11
Pic索引：9

总数量：432

https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=w6b/OC7ZkkQxjXOYkS6COfOA3FM8wSUCTuE74rL23eTuuKyt7z7JaPb2e5Q4r0M9nRaSzC3dZmboEyZGCTZQXGS/yUkBUvThK0HZgA1C4AGxzJCtT0EC3h5PFeAkVcMYsJ9MXbl4u4qQsP+xaLsnCdQCfVj8UUYFH/vcGamvfKrZu7Cn8K35UxPcsKOdKLB2pn3TNVkH4PWYkCUjotx4/w==  <->  [ok] - 剩余数量：410
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=jtzWrrt1uvs8DMWbvUSRP/XC2fI504HHdVo4tgUyKGjNbMBPsz78XNaJbO3JFZ1ObEHUQlckYE4Nr940p4e30JlpjLqJxiUyybkX50gqLN3gaqOveEiDkBzgNBE5AXOGt3TCYs9b4uHJB2bymKKm5IkwWUD6ZI5Qbl8Qg4knWkEXPVXTjaKJcPvMHUykoIflP8/YXmV/RM2ISW+u5FsC5w==  <->  [ok] - 剩余数量：409
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=w6b/OC7ZkkTIWZGhHakZ0/OA3FM8wSUCTuE74rL23eTuuKyt7z7JaCzwkH2nVdNpnRaSzC3dZmboEyZGCTZQXGS/yUkBUvThK0HZgA1C4AGxzJCtT0EC3h5PFeAkVcMYsJ9MXbl4u4qQsP+xaLsnCdQCfVj8UUYFH/vcGamvfKrZu7Cn8K35UxPcsKOdKLB2pn3TNVkH4PWYkCUjotx4/w==  <->  [ok] - 剩余数量：408
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=w6b/OC7ZkkS7JYI2hCV1vvOA3FM8wSUCTuE7

https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=1fYfBvKVda0iEU9P4UjjctWfI2U//f0XZ0E+s1drfbO6svKexixIoQi7MHc3Z77sYvfY+sUXOOXlUiMf62iPubgwXLHE9txgJv0YyvK/rWecX6DmzjK5LAPGgnfAzoJeeiNuk1H1nIsgiEVPfC+WFBiIabLNN0HoAxpCHIGe20o7DcAIwETg0xu0rLIjfR6m2RDao2eLRO5Vz7ykSKYgRw==  <->  [ok] - 剩余数量：381
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=qGLd4REoAwU/ROhYghfkAfOA3FM8wSUCTuE74rL23eSPAs7rPLuLEQXzMieMDkZdnRaSzC3dZmboEyZGCTZQXGS/yUkBUvThK0HZgA1C4AGxzJCtT0EC3h5PFeAkVcMYsJ9MXbl4u4qQsP+xaLsnCSAGwQ8w1KfyH/vcGamvfKrZu7Cn8K35UxPcsKOdKLB2gSCQgmUET1OYkCUjotx4/w==  <->  [ok] - 剩余数量：380
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gbgqpjC6W/UDQkGey1ygHWn9QsEmh2Oo5mSL48D9TzmhmMDBY5NJN/RYvfY+sUXOOXlUiMf62iPubgwXLHE9txgJv0YyvK/rWecX6DmzjK5LAPGgnfAzoJeeiNuk1H1nIsgiEVPfC+WFBiIabLNN0Hoqeu5hZlCRfM7DcAIwETg0xu0rLIjfR6m2RDao2eLRO4w95kWr0m2Ug==  <->  [ok] - 剩余数量：379
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gbUYgwyJolu8fXC2fI504HHdVo4tgUyKGjKEBo49uMb0taJbO3JFZ1

https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gak9urCBj5ixPOA3FM8wSUCTuE74rL23eRbGynfWvbOKCrETc6qKNCAnRaSzC3dZmboEyZGCTZQXGS/yUkBUvThK0HZgA1C4AGxzJCtT0EC3h5PFeAkVcMYsJ9MXbl4u4qQsP+xaLsnCfi8uaNCstSIH/vcGamvfKrZu7Cn8K35UxPcsKOdKLB2opR7V1Q/tFmYkCUjotx4/w==  <->  [ok] - 剩余数量：351
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gboV1d4zmvKHCYF4nkLByFFki+PA/U85obhH8pdqziJm2L32PrFFzjl5VIjH+toj7m4MFyxxPbcYCb9GMryv61nnF+g5s4yuSwDxoJ3wM6CXnojbpNR9ZyLIIhFT3wvlhQYiGmyzTdB6KnruYWZQkXzOw3ACMBE4NMbtKyyI30eptkQ2qNni0TumOdCV0ts30g=  <->  [ok] - 剩余数量：350
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gaWHzmASyyosvOA3FM8wSUCTuE74rL23eTo48hNg9WzOI5V+DR5hIPnnRaSzC3dZmboEyZGCTZQXGS/yUkBUvThK0HZgA1C4AGxzJCtT0EC3h5PFeAkVcMYsJ9MXbl4u4qQsP+xaLsnCfi8uaNCstSIH/vcGamvfKrZu7Cn8K35UxPcsKOdKLB2opR7V1Q/tFmYkCUjotx4/w==  <->  [ok] - 剩余数量：349
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gbxAPAQce9qK/OA3FM8wSUCTuE74rL23eRbGynfWvbOKDs50JAQetrjnRaSzC3dZmb

https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gbUAqDWs6yDGfOA3FM8wSUCTuE74rL23eSE6HF1NCA3mirETc6qKNCAnRaSzC3dZmboEyZGCTZQXGS/yUkBUvThK0HZgA1C4AGxzJCtT0EC3h5PFeAkVcMYsJ9MXbl4u4qQsP+xaLsnCfi8uaNCstSIH/vcGamvfKrZu7Cn8K35UxPcsKOdKLB2Pnf3JeJdXAiYkCUjotx4/w==  <->  [ok] - 剩余数量：322
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gYswf3LmLALcPOA3FM8wSUCTuE74rL23eSE6HF1NCA3mk6vJpe+GRQpnRaSzC3dZmboEyZGCTZQXGS/yUkBUvThK0HZgA1C4AGxzJCtT0EC3h5PFeAkVcMYsJ9MXbl4u4qQsP+xaLsnCfi8uaNCstSIH/vcGamvfKrZu7Cn8K35UxPcsKOdKLB2WS9HWhhMQwCYkCUjotx4/w==  <->  [ok] - 剩余数量：321
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gbWcpuJS//Sm/OA3FM8wSUCTuE74rL23eSE6HF1NCA3mr3Q8Zwq27HjnRaSzC3dZmboEyZGCTZQXGS/yUkBUvThK0HZgA1C4AGxzJCtT0EC3h5PFeAkVcMYsJ9MXbl4u4qQsP+xaLsnCfi8uaNCstSIH/vcGamvfKrZu7Cn8K35UxPcsKOdKLB23NmiWcJsuECYkCUjotx4/w==  <->  [ok] - 剩余数量：320
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gY022Rv4gLvAvOA3FM8wSUCTuE74rL23eSL3sr0ot9kYY/Tq7bbO1k

https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gY0Kq/jfT/LzM9+jCwx4n/NkkBfdKS4Ct/LxXGcXgziSGbivwdqsSBO7vYdmSbTx+UcNY5t5McX4raOUPJn/1IWEcpIOdX+99Ro2+h+YbZqvO7RQECWS0Bl7rk/OlEHVu1N3isKM+pDZCDvXEDdn0lYVZxgTKKcqp6IUqOyP9T0UCE+Hs6Rgp/hC/ANIk8Q0mrYyM3CLk6K9Q==  <->  [ok] - 剩余数量：294
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gbPVY5ZJ8ajTc9+jCwx4n/NkkBfdKS4Ct/LxXGcXgziSFH1ishVtjjw7vYdmSbTx+UcNY5t5McX4raOUPJn/1IWEcpIOdX+99Ro2+h+YbZqvO7RQECWS0Bl7rk/OlEHVu1N3isKM+pDZCDvXEDdn0lY2hDMk9h5rKWIUqOyP9T0UCE+Hs6Rgp/hC/ANIk8Q0mpDyItiJsvgnQ==  <->  [ok] - 剩余数量：293
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gY/+F51/DdHFPXC2fI504HHdVo4tgUyKGieFYlijxo30NaJbO3JFZ1ObEHUQlckYE4Nr940p4e30JlpjLqJxiUyybkX50gqLN3gaqOveEiDkBzgNBE5AXOGt3TCYs9b4uHJB2bymKKm5AACCcPXd7/Mbl8Qg4knWkEXPVXTjaKJcPvMHUykoIflwvU9BbN5TWCISW+u5FsC5w==  <->  [ok] - 剩余数量：292
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gaadl7fRaAvsM9+jCwx4n/NkkBfdKS4Ct/LxXGcXgziSNfaVTYy98O

...

https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gb6ZbnyxOf4jNWfI2U//f0XZ0E+s1drfbMv4iB6rYOJrfUCrTcGLlZIYvfY+sUXOOXlUiMf62iPubgwXLHE9txgJv0YyvK/rWecX6DmzjK5LAPGgnfAzoJeeiNuk1H1nIsgiEVPfC+WFBiIabLNN0Hoqeu5hZlCRfM7DcAIwETg0xu0rLIjfR6m2RDao2eLRO70bygyeISUrA==  <->  [ok] - 剩余数量：93
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gYdNK9Bwi9H0NWfI2U//f0XZ0E+s1drfbMv4iB6rYOJrSS389cFtTqgYvfY+sUXOOXlUiMf62iPubgwXLHE9txgJv0YyvK/rWecX6DmzjK5LAPGgnfAzoJeeiNuk1H1nIsgiEVPfC+WFBiIabLNN0HoI9kWyXUlSG47DcAIwETg0xu0rLIjfR6m2RDao2eLRO4axMRZN+g5FQ==  <->  [ok] - 剩余数量：92
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gYBXi+Mpg573NWfI2U//f0XZ0E+s1drfbMv4iB6rYOJrao8qRGEgUyPYvfY+sUXOOXlUiMf62iPubgwXLHE9txgJv0YyvK/rWecX6DmzjK5LAPGgnfAzoJeeiNuk1H1nIsgiEVPfC+WFBiIabLNN0Hoqeu5hZlCRfM7DcAIwETg0xu0rLIjfR6m2RDao2eLRO4oA4or5AWNog==  <->  [ok] - 剩余数量：91
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gYe4EmSfB6L1c9+jCwx4n/NkkBfdKS4Ct/LxXGcXgziSErhU1Nn+2HR7v

https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gad+GtTbkbw7NWfI2U//f0XZ0E+s1drfbMv4iB6rYOJrdoAg5WT8uLxYvfY+sUXOOXlUiMf62iPubgwXLHE9txgJv0YyvK/rWecX6DmzjK5LAPGgnfAzoJeeiNuk1H1nIsgiEVPfC+WFBiIabLNN0HoI9kWyXUlSG47DcAIwETg0xu0rLIjfR6m2RDao2eLRO45P5Vba3lioA==  <->  [ok] - 剩余数量：65
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2ganOpKMbnv1MIKvtc/TMhrkH7tbybWFAysWSaSu79Q7PWjUtDCPhk6f+HhC3Y37RC+dFpLMLd1mZugTJkYJNlBcZL/JSQFS9OErQdmADULgAbHMkK1PQQLeHk8V4CRVwxiwn0xduXi7ipCw/7FouycJ1AJ9WPxRRgUf+9wZqa98qtm7sKfwrflTE9ywo50osHYJxo11LycrzZiQJSOi3Hj/  <->  [ok] - 剩余数量：64
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gad+GtTbkbw7NWfI2U//f0XZ0E+s1drfbMv4iB6rYOJrdoAg5WT8uLxYvfY+sUXOOXlUiMf62iPubgwXLHE9txgJv0YyvK/rWecX6DmzjK5LAPGgnfAzoJeeiNuk1H1nIsgiEVPfC+WFBiIabLNN0HoI9kWyXUlSG47DcAIwETg0xu0rLIjfR6m2RDao2eLRO7VmqyZKcF8Nw==  <->  [ok] - 剩余数量：63
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gYjuBvD6TsoAsAEjVxcWN5nonf75/5qbZ8YgZ1nLwBOP32kHN

https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gY1SGeRQb8jNsAEjVxcWN5nonf75/5qbZ8YgZ1nLwBOPzv5uj21wp0+1ols7ckVnU5sQdRCVyRgTg2v3jSnh7fQmWmMuonGJTLJuRfnSCos3eBqo694SIOQHOA0ETkBc4a3dMJiz1vi4ckHZvKYoqbkiTBZQPpkjlBuXxCDiSdaQRc9VdONoolw+8wdTKSgh+X0/g4rHp3exohJb67kWwLn  <->  [ok] - 剩余数量：37
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gY1SGeRQb8jNsAEjVxcWN5nonf75/5qbZ8YgZ1nLwBOPzv5uj21wp0+1ols7ckVnU5sQdRCVyRgTg2v3jSnh7fQmWmMuonGJTLJuRfnSCos3eBqo694SIOQHOA0ETkBc4a3dMJiz1vi4ckHZvKYoqbkiTBZQPpkjlBuXxCDiSdaQRc9VdONoolw+8wdTKSgh+U9I2/YuusPF4hJb67kWwLn  <->  [ok] - 剩余数量：36
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gbFeU4XyVFXScAEjVxcWN5nonf75/5qbZ8YgZ1nLwBOP3tohrMhsrFo1ols7ckVnU5sQdRCVyRgTg2v3jSnh7fQmWmMuonGJTLJuRfnSCos3eBqo694SIOQHOA0ETkBc4a3dMJiz1vi4ckHZvKYoqbkiTBZQPpkjlBuXxCDiSdaQRc9VdONoolw+8wdTKSgh+V3dpLUAACHB4hJb67kWwLn  <->  [ok] - 剩余数量：35
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gbFeU4XyVFXScAEjVxcWN5nonf75/5qbZ

https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2ga/G6ElOFfwaMAEjVxcWN5nonf75/5qbZ8YgZ1nLwBOPx0g1IYDJQ5P1ols7ckVnU5sQdRCVyRgTg2v3jSnh7fQmWmMuonGJTLJuRfnSCos3eBqo694SIOQHOA0ETkBc4a3dMJiz1vi4ckHZvKYoqbkiTBZQPpkjlBuXxCDiSdaQRc9VdONoolw+8wdTKSgh+VNDi1+TokA44hJb67kWwLn  <->  [ok] - 剩余数量：8
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gaYitFsqBhtkMAEjVxcWN5nonf75/5qbZ8YgZ1nLwBOP5+FBPmj2zej1ols7ckVnU5sQdRCVyRgTg2v3jSnh7fQmWmMuonGJTLJuRfnSCos3eBqo694SIOQHOA0ETkBc4a3dMJiz1vi4ckHZvKYoqbkiTBZQPpkjlBuXxCDiSdaQRc9VdONoolw+8wdTKSgh+UIKrt0qgku54hJb67kWwLn  <->  [ok] - 剩余数量：7
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gap8azfpd8Y4NWfI2U//f0XZ0E+s1drfbMv4iB6rYOJrQtqehhbS4w9YvfY+sUXOOXlUiMf62iPubgwXLHE9txgJv0YyvK/rWecX6DmzjK5LAPGgnfAzoJeeiNuk1H1nIsgiEVPfC+WFBiIabLNN0HoI9kWyXUlSG47DcAIwETg0xu0rLIjfR6m2RDao2eLRO78QdqJ1y1t/A==  <->  [ok] - 剩余数量：6
https://extapi.smpcorp.com/ihgs/imagehandlergs.ashx?p1=fb86VxaQ2gaPzkrUaY2lKdWfI2U//f0XZ0E+s1drfbMv4iB6rYOJr


爬虫结束
